In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import KMeansSMOTE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report
from util.get_data import get_features_and_targets
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB, MultinomialNB

In [46]:
X, y = get_features_and_targets()

GENDER column transformed: [1 0]
EDUCATION_LEVEL column transformed: [2 1 3 4]
MARITAL_STATUS column transformed: [1 2 3]
CREDIT_LIMIT: 167 outliers capped.
AGE: 272 outliers capped.
SEPT_BILL: 2400 outliers capped.
AUG_BILL: 2395 outliers capped.
JULY_BILL: 2469 outliers capped.
JUNE_BILL: 2622 outliers capped.
MAY_BILL: 2725 outliers capped.
APRIL_BILL: 2693 outliers capped.
SEPT_PAYMENT: 2745 outliers capped.
AUG_PAYMENT: 2714 outliers capped.
JULY_PAYMENT: 2598 outliers capped.
JUNE_PAYMENT: 2994 outliers capped.
MAY_PAYMENT: 2945 outliers capped.
APRIL_PAYMENT: 2958 outliers capped.


In [47]:
type(X)

pandas.core.frame.DataFrame

In [48]:
X.head()

,CREDIT_LIMIT,GENDER,EDUCATION_LEVEL,MARITAL_STATUS,AGE,SEPT_PAY_STATUS,AUG_PAY_STATUS,JULY_PAY_STATUS,JUNE_PAY_STATUS,MAY_PAY_STATUS,...,JULY_BILL,JUNE_BILL,MAY_BILL,APRIL_BILL,SEPT_PAYMENT,AUG_PAYMENT,JULY_PAYMENT,JUNE_PAYMENT,MAY_PAYMENT,APRIL_PAYMENT
1,20000,1,2,1,24.0,2,2,-1,-1,-2,...,689.0,0.0,0.0,0.0,0,689.0,0.0,0.0,0,0.0
2,120000,1,2,2,26.0,-1,2,0,0,0,...,2682.0,3272.0,3455.0,3261.0,0,1000.0,1000.0,1000.0,0,2000.0
3,90000,1,2,2,34.0,0,0,0,0,0,...,13559.0,14331.0,14948.0,15549.0,1518,1500.0,1000.0,1000.0,1000,5000.0
4,50000,1,2,1,37.0,0,0,0,0,0,...,49291.0,28314.0,28959.0,29547.0,2000,2019.0,1200.0,1100.0,1069,1000.0
5,50000,0,2,1,57.0,-1,0,-1,0,0,...,35835.0,20940.0,19146.0,19131.0,2000,11250.5,10000.0,9000.0,689,679.0


In [49]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import KMeansSMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import numpy as np

# Define the transformation for numeric features
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Define the transformation for categorical features
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # Ensure output is dense
])

# Combine transformations into a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ], sparse_threshold=0)  # Ensures combined output is dense

# Define an explicit transformation to convert array to dense
def dense_transform(x):
    if isinstance(x, np.ndarray):
        return x
    else:
        return x.toarray()

# Build the pipeline
pipeline_gnb = ImbPipeline([
    ('preprocessor', preprocessor),
    ('to_dense', FunctionTransformer(dense_transform, accept_sparse=True)),  # Ensures data is dense
    ('smote', KMeansSMOTE(random_state=0, k_neighbors=5, cluster_balance_threshold=0.1)),
    ('classifier', GaussianNB())
])

# Now perform your train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit and score the pipeline
pipeline_gnb.fit(X_train, y_train)
score = pipeline_gnb.score(X_test, y_test)
print(f"Accuracy of GaussianNB model: {score}")


/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


Accuracy of GaussianNB model: 0.8073333333333333


/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [50]:
# Pipeline components for GaussianNB
numeric_transformer_gnb = Pipeline([
    ('scaler', StandardScaler())
])

preprocessor_gnb = ColumnTransformer([
    ('num', numeric_transformer_gnb, numeric_features)
], remainder='drop')  # Drop categorical features for GaussianNB

pipeline_gnb = ImbPipeline([
    ('preprocessor', preprocessor),
    ('to_dense', FunctionTransformer(dense_transform, accept_sparse=True)),  # Ensures data is dense
    ('smote', KMeansSMOTE(random_state=0, k_neighbors=5, cluster_balance_threshold=0.1)),
    ('classifier', GaussianNB())
])

In [51]:
# Pipeline components for MultinomialNB
categorical_transformer_mnb = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # Dense output for compatibility with SMOTE
])

preprocessor_mnb = ColumnTransformer([
    ('cat', categorical_transformer_mnb, categorical_features)
], remainder='drop')  # Drop numeric features for MultinomialNB

pipeline_mnb = ImbPipeline([
    ('preprocessor', preprocessor_mnb),
    ('smote', KMeansSMOTE(random_state=0)),
    ('classifier', MultinomialNB())
])


In [52]:
from sklearn.model_selection import train_test_split

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train both pipelines
pipeline_gnb.fit(X_train, y_train)
pipeline_mnb.fit(X_train, y_train)


/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklea

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['GENDER', 'EDUCATION_LEVEL',
                                                   'MARITAL_STATUS',
                                                   'SEPT_PAY_STATUS',
                                                   'AUG_PAY_STATUS',
                                                   'JULY_PAY_STATUS',
                                                   'JUNE_PAY_STATUS',
                                                   'MAY_PAY_STATUS',
                                                   'APRIL_PAY_STATUS'])])),
                ('smote', KMeansSMOTE(random_state=0)),
                ('classifier', MultinomialNB())])

In [53]:
import numpy as np

# Get probability predictions
probs_gnb = pipeline_gnb.predict_proba(X_test)
probs_mnb = pipeline_mnb.predict_proba(X_test)

# Average the probabilities from both models
avg_probs = (probs_gnb + probs_mnb) / 2

# Make final predictions based on the average probabilities
final_predictions = np.argmax(avg_probs, axis=1)

# Evaluate the combined model
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, final_predictions)
print(f"Combined Accuracy of GaussianNB and MultinomialNB: {accuracy}")


Combined Accuracy of GaussianNB and MultinomialNB: 0.8098333333333333


In [54]:
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=0)

# Initialize lists to store indices for each fold
train_indices = []
validation_indices = []

# Split data into folds
for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    train_indices.append(train_idx)
    validation_indices.append(val_idx)
    print(f"Fold {fold + 1}:")
    print(f"  Training indices: {train_idx[:5]}... ({len(train_idx)} samples)")
    print(f"  Validation indices: {val_idx[:5]}... ({len(val_idx)} samples)")

Fold 1:
  Training indices: [0 1 2 3 5]... (24000 samples)
  Validation indices: [ 4  6  7 18 22]... (6000 samples)
Fold 2:
  Training indices: [0 1 2 3 4]... (24000 samples)
  Validation indices: [ 8  9 12 16 17]... (6000 samples)
Fold 3:
  Training indices: [4 6 7 8 9]... (24000 samples)
  Validation indices: [0 1 2 3 5]... (6000 samples)
Fold 4:
  Training indices: [0 1 2 3 4]... (24000 samples)
  Validation indices: [11 21 23 24 26]... (6000 samples)
Fold 5:
  Training indices: [0 1 2 3 4]... (24000 samples)
  Validation indices: [10 13 19 28 40]... (6000 samples)


In [55]:
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import KFold

# Assuming X and y are your complete dataset
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Store scores for averaging
train_scores = []
validation_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\nFold {fold + 1}/{n_splits}")

    # Split dataset into training and validation sets
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # Ensure y_train and y_val are 1-dimensional if y is a DataFrame with a single target column
    # These will convert DataFrame columns to 1D numpy arrays
    y_train = y_train.squeeze().values
    y_val = y_val.squeeze().values

    # Fit both pipelines on the training data
    pipeline_gnb.fit(X_train, y_train)
    pipeline_mnb.fit(X_train, y_train)

    # Predict on the training data
    y_train_pred_gnb = pipeline_gnb.predict(X_train)
    y_train_pred_mnb = pipeline_mnb.predict(X_train)

    # Average the training predictions
    avg_train_predictions = (y_train_pred_gnb + y_train_pred_mnb) / 2
    avg_train_predictions = np.round(avg_train_predictions).astype(int)  # Convert averages to discrete class predictions

    # Compute training accuracy
    train_accuracy = accuracy_score(y_train, avg_train_predictions)
    train_scores.append(train_accuracy)

    # Predict probabilities on the validation set and combine predictions
    probs_gnb = pipeline_gnb.predict_proba(X_val)
    probs_mnb = pipeline_mnb.predict_proba(X_val)
    avg_probs = (probs_gnb + probs_mnb) / 2
    final_predictions = np.argmax(avg_probs, axis=1)

    # Compute and append validation accuracy
    validation_accuracy = accuracy_score(y_val, final_predictions)
    validation_scores.append(validation_accuracy)

    print(f"Training Accuracy for current fold: {train_accuracy:.2f}")
    print(f"Validation Accuracy for current fold: {validation_accuracy:.2f}")

# Calculate average scores across all folds
average_train_score = np.mean(train_scores)
average_val_score = np.mean(validation_scores)

print("\nK-Fold Cross-Validation Results for Ensemble Naive Bayes Model:")
print(f"Average Training Accuracy: {average_train_score:.2f}")
print(f"Average Validation Accuracy: {average_val_score:.2f}")


/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)



Fold 1/5


/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


Training Accuracy for current fold: 0.81
Validation Accuracy for current fold: 0.81

Fold 2/5


/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


...

Training Accuracy for current fold: 0.80
Validation Accuracy for current fold: 0.82

Fold 4/5


/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


Training Accuracy for current fold: 0.80
Validation Accuracy for current fold: 0.79

Fold 5/5


/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


Training Accuracy for current fold: 0.81
Validation Accuracy for current fold: 0.81

K-Fold Cross-Validation Results for Ensemble Naive Bayes Model:
Average Training Accuracy: 0.80
Average Validation Accuracy: 0.81


In [56]:
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import KFold

# Assuming X and y are the complete datasets
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Store scores for averaging
gnb_train_scores = []
gnb_validation_scores = []
mnb_train_scores = []
mnb_validation_scores = []

ensemble_train_scores = []
ensemble_validation_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\nFold {fold + 1}/{n_splits}")

    # Split dataset into training and validation sets
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx].squeeze().values, y.iloc[val_idx].squeeze().values

    # Fit both pipelines on the training data
    pipeline_gnb.fit(X_train, y_train)
    pipeline_mnb.fit(X_train, y_train)

    # Predict on the training data
    y_train_pred_gnb = pipeline_gnb.predict(X_train)
    y_train_pred_mnb = pipeline_mnb.predict(X_train)

    # Calculate individual training accuracies
    gnb_train_accuracy = accuracy_score(y_train, y_train_pred_gnb)
    mnb_train_accuracy = accuracy_score(y_train, y_train_pred_mnb)
    gnb_train_scores.append(gnb_train_accuracy)
    mnb_train_scores.append(mnb_train_accuracy)

    # Average the training predictions for ensemble
    avg_train_predictions = (y_train_pred_gnb + y_train_pred_mnb) / 2
    avg_train_predictions = np.round(avg_train_predictions).astype(int)
    ensemble_train_accuracy = accuracy_score(y_train, avg_train_predictions)
    ensemble_train_scores.append(ensemble_train_accuracy)

    # Predict probabilities on the validation set
    probs_gnb = pipeline_gnb.predict_proba(X_val)
    probs_mnb = pipeline_mnb.predict_proba(X_val)
    
    # Calculate individual validation accuracies
    y_val_pred_gnb = np.argmax(probs_gnb, axis=1)
    y_val_pred_mnb = np.argmax(probs_mnb, axis=1)
    gnb_val_accuracy = accuracy_score(y_val, y_val_pred_gnb)
    mnb_val_accuracy = accuracy_score(y_val, y_val_pred_mnb)
    gnb_validation_scores.append(gnb_val_accuracy)
    mnb_validation_scores.append(mnb_val_accuracy)
    
    # Average the validation probabilities for ensemble
    avg_probs = (probs_gnb + probs_mnb) / 2
    final_predictions = np.argmax(avg_probs, axis=1)
    ensemble_validation_accuracy = accuracy_score(y_val, final_predictions)
    ensemble_validation_scores.append(ensemble_validation_accuracy)

    # Output fold results
    print(f"GNB Training Accuracy: {gnb_train_accuracy:.2f}, Validation Accuracy: {gnb_val_accuracy:.2f}")
    print(f"MNB Training Accuracy: {mnb_train_accuracy:.2f}, Validation Accuracy: {mnb_val_accuracy:.2f}")
    print(f"Ensemble Training Accuracy: {ensemble_train_accuracy:.2f}, Validation Accuracy: {ensemble_validation_accuracy:.2f}")

# Calculate average scores across all folds
average_gnb_train = np.mean(gnb_train_scores)
average_gnb_val = np.mean(gnb_validation_scores)
average_mnb_train = np.mean(mnb_train_scores)
average_mnb_val = np.mean(mnb_validation_scores)

average_ensemble_train = np.mean(ensemble_train_scores)
average_ensemble_val = np.mean(ensemble_validation_scores)

print("\nFinal K-Fold Cross-Validation Results:")
print(f"GNB Average Training Accuracy: {average_gnb_train:.2f}, Average Validation Accuracy: {average_gnb_val:.2f}")
print(f"MNB Average Training Accuracy: {average_mnb_train:.2f}, Average Validation Accuracy: {average_mnb_val:.2f}")
print(f"Ensemble Average Training Accuracy: {average_ensemble_train:.2f}, Average Validation Accuracy: {average_ensemble_val:.2f}")



Fold 1/5


/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


GNB Training Accuracy: 0.81, Validation Accuracy: 0.81
MNB Training Accuracy: 0.80, Validation Accuracy: 0.80
Ensemble Training Accuracy: 0.81, Validation Accuracy: 0.81

Fold 2/5


/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


GNB Training Accuracy: 0.80, Validation Accuracy: 0.80
MNB Training Accuracy: 0.80, Validation Accuracy: 0.80
Ensemble Training Accuracy: 0.80, Validation Accuracy: 0.81

Fold 3/5


...

GNB Training Accuracy: 0.80, Validation Accuracy: 0.81
MNB Training Accuracy: 0.80, Validation Accuracy: 0.81
Ensemble Training Accuracy: 0.80, Validation Accuracy: 0.82

Fold 4/5


/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


GNB Training Accuracy: 0.80, Validation Accuracy: 0.79
MNB Training Accuracy: 0.81, Validation Accuracy: 0.80
Ensemble Training Accuracy: 0.80, Validation Accuracy: 0.79

Fold 5/5


/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/Users/arecalde/anaconda3/envs/ds_course_lessons/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


GNB Training Accuracy: 0.81, Validation Accuracy: 0.80
MNB Training Accuracy: 0.81, Validation Accuracy: 0.80
Ensemble Training Accuracy: 0.81, Validation Accuracy: 0.81

Final K-Fold Cross-Validation Results:
GNB Average Training Accuracy: 0.80, Average Validation Accuracy: 0.80
MNB Average Training Accuracy: 0.80, Average Validation Accuracy: 0.80
Ensemble Average Training Accuracy: 0.80, Average Validation Accuracy: 0.81
